In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pickle

from custom_transform import custom_transform

In [2]:
data=pd.read_csv('train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

/tmp/ipykernel_114032/1690689035.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(), inplace=True)
/tmp/ipykernel_114032/1690689035.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [5]:
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn import set_config
set_config(transform_output='pandas')


preprocessor_1 = FunctionTransformer(custom_transform, validate=False)

numerical_features = ['Age', 'Fare', 'Family']
categorical_features = ['Sex', 'Embarked']

preprocessor_2 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
    ], remainder='passthrough'
)

preprocessing_pipeline=Pipeline([
    ('feature_engineering',preprocessor_1), 
    ('scaling_encoding',preprocessor_2)
])


model= LogisticRegression()


clf= Pipeline([('preprocessing', preprocessing_pipeline),
               ('model', model)
        ])




In [7]:
y= data['Survived']
X = data.drop('Survived', axis=1)

X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
clf.fit(X_train, y_train)


/home/somya/miniconda3/envs/models/lib/python3.10/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('feature_engineering',
                                  FunctionTransformer(func=<function custom_transform at 0x79e18e85f1c0>)),
                                 ('scaling_encoding',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('num',
                                                                   StandardScaler(),
                                                                   ['Age',
                                                                    'Fare',
                                                                    'Family']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False),
                                                                   ['Sex',
                                                                    'Embarked'])]))])),
                ('model', LogisticRegression())])

In [9]:
y_pred = clf.predict(X_test)
acc_test= accuracy_score(y_test, y_pred)

print(f"Testing Accuracy: {acc_test}")

Testing Accuracy: 0.8044692737430168


In [10]:
a=np.array([8,1,'Futrelle, Mrs.','male',31.0,1,0,113803,53.1000,'C123','S'])
a=a.reshape(1, -1)
a_df = pd.DataFrame(a, columns=X.columns)

ans= clf.predict(a_df)

In [11]:
ans

array([0])

In [12]:
a_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,8,1,"Futrelle, Mrs.",male,31.0,1,0,113803,53.1,C123,S


In [13]:
#deployment
import pickle

In [14]:
pickle.dump(clf, open('model.pkl', 'wb'))

In [15]:
pkld_mdl=pickle.load(open('model.pkl', 'rb'))

In [16]:
pkld_mdl.predict(a_df)

array([0])

In [17]:
print(np.__version__)

2.0.1
